# Basics Understanding ->

BASIC WORK-FLOW -->

create an engine to connect with the database file/server

create a connection using the engine.

execute commands with the connection.






importing everything

In [285]:
import sqlalchemy as sqla
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base


from sqlalchemy.orm import relationship

AlchemyBase = declarative_base()

## Defining the schema for our ORM

In [286]:
class Course(AlchemyBase):
    __tablename__ = 'course'
    course_id = Column(Integer, primary_key=True, autoincrement=True)
    course_code = Column(String, unique=True, nullable=False)
    course_name = Column(String, nullable=False)
    course_description = Column(String)


class Student(AlchemyBase):
    __tablename__ = 'student'
    student_id = Column(Integer, primary_key=True, autoincrement=True)
    roll_number = Column(String, unique=True, nullable=False)
    first_name = Column(String, nullable=False)
    last_name = Column(String)


class Enrolments(AlchemyBase):
    __tablename__ = 'enrollments'
    enrollment_id = Column(Integer, primary_key=True, autoincrement=True)
    estudent_id = Column(Integer, ForeignKey(Student.student_id), nullable=False)
    ecourse_id = Column(Integer, ForeignKey(Course.course_id), nullable=False)


In [287]:
import sqlalchemy as db
from sqlalchemy import *



In [288]:
engine = create_engine("sqlite:///database.sqlite3")

In [289]:
connection = engine.connect()

In [290]:
Session = db.orm.sessionmaker(bind=engine)

session = Session()



In [291]:
print(engine.table_names());

['course', 'enrollments', 'sqlite_sequence', 'student']


/var/folders/6j/v9zb038n2bxdy2_vbd43mhnc0000gn/T/ipykernel_66096/1320240585.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names());


In [292]:
def print_results(query_to_execute):
    columns = connection.execute(query_to_execute).keys()
    results = connection.execute(query_to_execute)

    for column in columns:
        print(column, end=" | ")
    print("\n---------------------------------------------------------------")

    for result in results.fetchall(): # No need to explicitly mention fetchall()
        for data in result:
            print(data, end="     |      ")
        print()



In [297]:
result = connection.execute("SELECT course.course_id, course.course_code, course.course_name, course.course_description FROM course")

for r in result:
    print(r)

(1, 'CSE01', 'MAD I', 'Modern Application Development - I')
(2, 'CSE02', 'DBMS', 'Database management Systems')
(3, 'CSE03', 'PDSA', 'Programming, Data Structures and Algorithms using Python')
(4, 'BST13', 'BDM', 'Business Data Management')
(7, 'PG001', 'JAVA', 'Programming concepts using JAVA')


In [296]:

query = select(Course)
print("-----------")
print(query)
print("-----------")
print_results(query)

-----------
SELECT course.course_id, course.course_code, course.course_name, course.course_description 
FROM course
-----------
course_id | course_code | course_name | course_description | 
---------------------------------------------------------------
1     |      CSE01     |      MAD I     |      Modern Application Development - I     |      
2     |      CSE02     |      DBMS     |      Database management Systems     |      
3     |      CSE03     |      PDSA     |      Programming, Data Structures and Algorithms using Python     |      
4     |      BST13     |      BDM     |      Business Data Management     |      
7     |      PG001     |      JAVA     |      Programming concepts using JAVA     |      


# WHERE CLAUSE

In [299]:
query = select(Course).where(Course.course_name == 'MAD I')

In [300]:
print_results(query)

course_id | course_code | course_name | course_description | 
---------------------------------------------------------------
1     |      CSE01     |      MAD I     |      Modern Application Development - I     |      


# AND / OR / NOT conditions in WHERE clause

## OR

In [301]:
query = select(Course).where(or_(Course.course_name == 'MAD I', Course.course_name == 'PDSA', Course.course_name == 'BDM'))
print_results(query)

course_id | course_code | course_name | course_description | 
---------------------------------------------------------------
1     |      CSE01     |      MAD I     |      Modern Application Development - I     |      
3     |      CSE03     |      PDSA     |      Programming, Data Structures and Algorithms using Python     |      
4     |      BST13     |      BDM     |      Business Data Management     |      


## AND

In [169]:
query = select(Student).where(Student.last_name == 'Ruhil')

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
19     |      102     |      Pushpak     |      Ruhil     |      
20     |      103     |      XYZ     |      Ruhil     |      
21     |      104     |      PQR     |      Ruhil     |      
22     |      105     |      ABC     |      Ruhil     |      


In [302]:
query = select(Student).where(and_(Student.first_name == 'ABC', Student.last_name == 'Ruhil'))

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
22     |      105     |      ABC     |      Ruhil     |      


## NOT

In [173]:
query = select(Student).where(not_(Student.roll_number == 103))

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
18     |      101     |      Pushpak     |           |      
19     |      102     |      Pushpak     |      Ruhil     |      
21     |      104     |      PQR     |      Ruhil     |      
22     |      105     |      ABC     |      Ruhil     |      


# IN

In [174]:
query = select(Student).where(Student.first_name.in_(["ABC", "XYZ"]))

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
20     |      103     |      XYZ     |      Ruhil     |      
22     |      105     |      ABC     |      Ruhil     |      


# NOT IN

In [179]:
query = select(Student).where(Student.first_name.notin_(["Pushpak", "XYZ"]))

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
21     |      104     |      PQR     |      Ruhil     |      
22     |      105     |      ABC     |      Ruhil     |      


In [182]:
query = select(Student).order_by(asc(Student.roll_number))

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
18     |      101     |      Pushpak     |           |      
19     |      102     |      Pushpak     |      Ruhil     |      
20     |      103     |      XYZ     |      Ruhil     |      
21     |      104     |      PQR     |      Ruhil     |      
22     |      105     |      ABC     |      Ruhil     |      


In [183]:
query = select(Student).order_by(desc(Student.roll_number))

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
22     |      105     |      ABC     |      Ruhil     |      
21     |      104     |      PQR     |      Ruhil     |      
20     |      103     |      XYZ     |      Ruhil     |      
19     |      102     |      Pushpak     |      Ruhil     |      
18     |      101     |      Pushpak     |           |      


# Using SQL Functions and Group by

In [193]:
query = select(func.count(Student.first_name).label("Student Count"), Student.first_name).group_by(Student.first_name).order_by(desc("Student Count"))

print_results(query)

Student Count | first_name | 
---------------------------------------------------------------
2     |      Pushpak     |      
1     |      XYZ     |      
1     |      PQR     |      
1     |      ABC     |      


In [194]:
query = select(func.sum(Student.roll_number).label("Sum of Roll"))

print_results(query)

Sum of Roll | 
---------------------------------------------------------------
515     |      


# Distinct

In [196]:
query = select(Student.first_name.distinct())

print_results(query)

first_name | 
---------------------------------------------------------------
Pushpak     |      
XYZ     |      
PQR     |      
ABC     |      


In [201]:
query = select(Student.first_name.distinct().label("Distinct Names"), "*")

print_results(query)

Distinct Names | student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
Pushpak     |      18     |      101     |      Pushpak     |           |      
Pushpak     |      19     |      102     |      Pushpak     |      Ruhil     |      
XYZ     |      20     |      103     |      XYZ     |      Ruhil     |      
PQR     |      21     |      104     |      PQR     |      Ruhil     |      
ABC     |      22     |      105     |      ABC     |      Ruhil     |      


# Limit

In [214]:
query = select(Student).order_by(desc(Student.roll_number)).limit(2)

print_results(query)

student_id | roll_number | first_name | last_name | 
---------------------------------------------------------------
22     |      105     |      ABC     |      Ruhil     |      
21     |      104     |      PQR     |      Ruhil     |      


# Join

In [219]:
query = select(Student, Enrolments).where(Student.student_id == Enrolments.estudent_id)

print_results(query)

student_id | roll_number | first_name | last_name | enrollment_id | estudent_id | ecourse_id | 
---------------------------------------------------------------
18     |      101     |      Pushpak     |           |      31     |      18     |      1     |      
18     |      101     |      Pushpak     |           |      32     |      18     |      2     |      


In [231]:
query = select(Student, Enrolments).select_from(join(Student, Enrolments))

print_results(query)

student_id | roll_number | first_name | last_name | enrollment_id | estudent_id | ecourse_id | 
---------------------------------------------------------------
18     |      101     |      Pushpak     |           |      31     |      18     |      1     |      
18     |      101     |      Pushpak     |           |      32     |      18     |      2     |      


# Inserting data

In [232]:
data_to_add = Student(roll_number=120, first_name='Ajay', last_name='Singh')

In [234]:

Session = db.orm.sessionmaker(bind=engine)

session = Session()



In [235]:
session.add(data_to_add)

In [236]:
session.commit()

In [243]:
data_not_to_add = Student(roll_number=121, first_name='Ajay2', last_name='Singh2')
session.add(data_not_to_add)


In [244]:
session.rollback()

In [241]:
session.commit()

Using the INSERT/VALUE command methods

In [250]:

query = insert(Course).values(course_name='JAVA', course_code='PROG1', course_description='Programming concepts using JAVA')

connection.execute(query)


# UPDATE

In [251]:

query = update(Course).where(Course.course_name == 'JAVA').values(course_code = 'PG001')

In [254]:
connection.execute(query)


# QUERY API

In [269]:
courses = session.query(Course).all()
for course in courses:
    print(course.course_id, course.course_code, course.course_name, course.course_description, sep="   |   " )

1   |   CSE01   |   MAD I   |   Modern Application Development - I
2   |   CSE02   |   DBMS   |   Database management Systems
3   |   CSE03   |   PDSA   |   Programming, Data Structures and Algorithms using Python
4   |   BST13   |   BDM   |   Business Data Management
7   |   PG001   |   JAVA   |   Programming concepts using JAVA


In [281]:
courses = session.query(Course).filter_by(course_name="JAVA")
for course in courses:
    print(course.course_id, course.course_code, course.course_name, course.course_description, sep="   |   " )

7   |   PG001   |   JAVA   |   Programming concepts using JAVA
